In [1]:
import memory as pm
import old_memory as om
from collections import namedtuple
import torch
import time

import numpy as np

%load_ext autoreload

%autoreload 2

In [2]:
Arguments = namedtuple(
    'Arguments', 'device history_length discount multi_step priority_weight priority_exponent')
device = torch.device('cuda')
history_length=4
discount=0.99
multi_step=3
priority_weight=0.4
priority_exponent=0.5

args=Arguments(device, history_length, discount, multi_step, priority_weight, priority_exponent)

base_size = 2**13

print(args.device)


cuda


In [3]:
def generate_transition():
    state = torch.rand(size=(3, 84, 84))
    action = np.random.randint(10)
    reward = np.random.uniform()
    terminal = action > 8
    return state, action, reward, terminal

def generate_tree_data():
    data = (np.random.randint(10), np.ones((84, 84), dtype=np.uint8), np.random.randint(10), 0.0, False)
    priority = np.random.uniform(high=2.0)
    return data, priority

In [4]:
# Test parallel segment tree v old segment tree
pst = pm.SegmentTree(int(1e5))
ost = om.SegmentTree(int(1e5))

In [5]:
for i in range(1000): # Add random data
    t = generate_tree_data()
    pst.append(*t)
    ost.append(*t)

In [6]:
# Check total match
print(pst.total())
print(ost.total())


1034.616
1034.616


In [8]:
# Check and time finding values

# Parallel tree
s_t = time.time()
vals, idx_d, st_idx = pst.find(np.array([i*950/256 for i in range(256)])+1e-8)
e_t = time.time()
print(vals)
print(idx_d)
print(st_idx)
print('Time taken {}'.format(e_t-s_t))

[1.0402731  0.222832   0.9384391  1.7385993  0.5447644  1.1748855
 0.68893373 0.46289703 1.8510692  1.2963842  1.3397082  1.7713443
 1.5922421  1.8539671  1.4067364  0.84001553 1.0812905  1.5972332
 1.3901     1.0061234  1.5087178  1.9691579  1.7055875  1.8004625
 1.807829   1.6092045  1.9937223  1.2652521  1.1631775  0.78137434
 1.5668952  1.7946529  1.8508635  1.8072838  0.80279124 1.3363489
 0.28317347 1.6210195  1.1535898  0.16679026 0.23373102 1.9564984
 1.0365139  0.2256025  1.1470875  1.770508   1.9696428  1.4208353
 1.4357722  1.3779279  1.9548326  1.8820432  1.9446081  1.9489532
 0.90649354 0.83796495 0.8903082  1.9211117  1.6495525  1.2663505
 1.6932904  0.617352   1.5642105  1.7526863  1.1830211  1.4726901
 1.2805132  1.2764714  0.51142687 0.5227238  1.3968081  1.0308181
 0.9841133  0.4743872  1.6463034  0.6840366  1.5056144  1.7696966
 1.5266981  0.8740722  1.960559   1.359344   0.43648654 1.8445967
 1.2425773  1.9128623  1.9532284  1.4386253  0.6940388  0.7428471
 1.803380

In [9]:
# Old tree
s_t = time.time()
vals, idx_d, idx = zip(*[ost.find(i*950/256+1e-8) for i in range(256)])
e_t = time.time()
print(vals)
print(idx_d)
print(idx)
print('Time taken {}'.format(e_t-s_t))

(1.0402731, 0.222832, 0.9384391, 1.7385993, 0.5447644, 1.1748855, 0.68893373, 0.46289703, 1.8510692, 1.2963842, 1.3397082, 1.7713443, 1.5922421, 1.8539671, 1.4067364, 0.84001553, 1.0812905, 1.5972332, 1.3901, 1.0061234, 1.5087178, 1.9691579, 1.7055875, 1.8004625, 1.807829, 1.6092045, 1.9937223, 1.2652521, 1.1631775, 0.78137434, 1.5668952, 1.7946529, 1.8508635, 1.8072838, 0.80279124, 1.3363489, 0.28317347, 1.6210195, 1.1535898, 0.16679026, 0.23373102, 1.9564984, 1.0365139, 0.2256025, 1.1470875, 1.770508, 1.9696428, 1.4208353, 1.4357722, 1.3779279, 1.9548326, 1.8820432, 1.9446081, 1.9489532, 0.90649354, 0.83796495, 0.8903082, 1.9211117, 1.6495525, 1.2663505, 1.6932904, 0.617352, 1.5642105, 1.7526863, 1.1830211, 1.4726901, 1.2805132, 1.2764714, 0.51142687, 0.5227238, 1.3968081, 1.0308181, 0.9841133, 0.4743872, 1.6463034, 0.6840366, 1.5056144, 1.7696966, 1.5266981, 0.8740722, 1.960559, 1.359344, 0.43648654, 1.8445967, 1.2425773, 1.9128623, 1.9532284, 1.4386253, 0.6940388, 0.7428471, 1.8033

In [10]:
# Time updating values

# Parallel tree
s_t = time.time()
pst.update(st_idx[np.arange(200)], np.arange(200))
e_t = time.time()
print('Time taken {}'.format(e_t-s_t))

Time taken 0.002521991729736328


In [11]:
# Old tree
s_t = time.time()
[ost.update(idx[i], i) for i in range(200)]
e_t = time.time()
print('Time taken {}'.format(e_t-s_t))

Time taken 0.001809835433959961


In [12]:
# Test parallel v old replay memories

base_size = 2**13 # Deterministic behaviour is identical if the memory sizes are multiples of 2, otherwise the old replay memory will start adding values from a different tree level

prm = pm.ReplayMemory(args, int(base_size))
orm = om.ReplayMemory(args, int(base_size))

In [13]:
for i in range(3000): # Add random data
    t = generate_transition()
    prm.append(*t)
    orm.append(*t)

In [15]:
# Check sum

print(prm.transitions.total())
print(orm.transitions.total())

55469.57
55469.57


In [16]:
# Check index

print(prm.transitions.index)
print(orm.transitions.index)

808

In [10]:
# Time samplings

# Parallel memory
s_t = time.time()
p_out = prm.sample_det(256)
e_t = time.time()
print(e_t-s_t)

[  11   35   58   82  105  128  152  175  199  222  246  269  292  316
  339  363  386  410  433  457  480  503  527  550  574  597  621  644
  667  691  714  738  761  785  808  832  855  878  902  925  949  972
  996 1019 1042 1066 1089 1113 1136 1160 1183 1207 1230 1253 1277 1300
 1324 1347 1371 1394 1417 1441 1464 1488 1511 1535 1558 1582 1605 1628
 1652 1675 1699 1722 1746 1769 1792 1816 1839 1863 1886 1910 1933 1957
 1980 2003 2027 2050 2074 2097 2121 2144 2167 2191 2214 2238 2261 2285
 2308 2332 2355 2378 2402 2425 2449 2472 2496 2519 2542 2566 2589 2613
 2636 2660 2683 2707 2730 2753 2777 2800 2824 2847 2871 2894 2917 2941
 2964 2988 3011 3035 3058 3082 3105 3128 3152 3175 3199 3222 3246 3269
 3292 3316 3339 3363 3386 3410 3433 3457 3480 3503 3527 3550 3574 3597
 3621 3644 3667 3691 3714 3738 3761 3785 3808 3832 3855 3878 3902 3925
 3949 3972 3996 4019 4042 4066 4089 4113 4136 4160 4183 4207 4230 4253
 4277 4300 4324 4347 4371 4394 4417 4441 4464 4488 4511 4535 4558 4582
 4605 

In [11]:
# Old memory
s_t = time.time()
out = orm.sample_det(256)
e_t = time.time()
print(e_t-s_t)

(11, 35, 58, 82, 105, 128, 152, 175, 199, 222, 246, 269, 292, 316, 339, 363, 386, 410, 433, 457, 480, 503, 527, 550, 574, 597, 621, 644, 667, 691, 714, 738, 761, 785, 808, 832, 855, 878, 902, 925, 949, 972, 996, 1019, 1042, 1066, 1089, 1113, 1136, 1160, 1183, 1207, 1230, 1253, 1277, 1300, 1324, 1347, 1371, 1394, 1417, 1441, 1464, 1488, 1511, 1535, 1558, 1582, 1605, 1628, 1652, 1675, 1699, 1722, 1746, 1769, 1792, 1816, 1839, 1863, 1886, 1910, 1933, 1957, 1980, 2003, 2027, 2050, 2074, 2097, 2121, 2144, 2167, 2191, 2214, 2238, 2261, 2285, 2308, 2332, 2355, 2378, 2402, 2425, 2449, 2472, 2496, 2519, 2542, 2566, 2589, 2613, 2636, 2660, 2683, 2707, 2730, 2753, 2777, 2800, 2824, 2847, 2871, 2894, 2917, 2941, 2964, 2988, 3011, 3035, 3058, 3082, 3105, 3128, 3152, 3175, 3199, 3222, 3246, 3269, 3292, 3316, 3339, 3363, 3386, 3410, 3433, 3457, 3480, 3503, 3527, 3550, 3574, 3597, 3621, 3644, 3667, 3691, 3714, 3738, 3761, 3785, 3808, 3832, 3855, 3878, 3902, 3925, 3949, 3972, 3996, 4019, 4042, 4066, 40

In [33]:
# Check output samplings are equivalent
for p_elem, elem in zip(p_out[1:], out[1:]):
    print(((p_elem - elem) < 1e-5).all())

tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')


In [269]:
# Time updates

# Parallel memory
s_t = time.time()
prm.update_priorities(p_out[0], np.arange(256))
e_t = time.time()
print(e_t-s_t)

In [ ]:
# Old memory
s_t = time.time()
orm.update_priorities(out[0], np.arange(256))
e_t = time.time()
print(e_t-s_t)

In [24]:
# Check all states are equivalent

for state, p_state in zip(orm, prm):
    print(((state - p_state) < 1e-5).all())

tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(Tru

tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(Tru

tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(Tru

tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(Tru

tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(Tru

...

tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(Tru

tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(Tru

tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(Tru

tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(Tru

tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(Tru